In [17]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import os
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Reading the table containing Headlines from html file

In [5]:
html_tables = {}

for file_name in os.listdir('datasets'):
    file_path = f'datasets/{file_name}'
    file_object = open(file_path,'r')
    html = BeautifulSoup(file_object)
    # finding the headlines table which has id news-table
    html_table = html.find(id = 'news-table')
    # Adding the table name to the dictionary
    html_tables[file_name] = html_table

In [6]:
#html_tables

## Exploring the data from the headlines table

In [7]:
html_tables.keys()

dict_keys(['fb_05ene.html', 'fb_22sep.html', 'tsla_05ene.html', 'tsla_22sep.html', 'tsla_26nov.html'])

In [11]:
# Reading facebook headlines from sep 22nd
fb = html_tables['fb_22sep.html']

# Reading the <tr> in the above table
fb_tr = fb.findAll('tr')

for i, table_row in enumerate(fb_tr):
    # Reading the data from <td> tag
    data_text = table_row.td.get_text()
    
    # Reading data from <a> tag
    link_text = table_row.a.get_text()
    
    #Printing the first 5 headines from the table
    print(f'{i}:')
    print(data_text)
    print(link_text)
    
    # Breaking the loop after 5 iterations
    if i == 5:
        break

0:
Sep-22-18 11:08AM  
How former Facebook and Google engineer Bret Taylor earned Marc Benioff's trust at Salesforce
1:
12:02AM  
White House Drafts Order To Look Into Google, Facebook Practices
2:
Sep-21-18 06:21PM  
Facebook Withdraws Direct Promotion of Political Campaigns
3:
06:18PM  
Facebook's Plan to Pull Back Campaign Support to Trump in 2020
4:
06:17PM  
Facebook's Plan to Pull Back Campaign Support to Trump in 2020
5:
06:10PM  
Does F.A.N.G. still stand? 8 trades


## Extracting Headlines along with company name, date and time

In [14]:
parsed_news = []

# Going over HTML table
for file_name, news_table in html_tables.items():
    # Iterating over teach <tr> tag and extracting headlines(from <a>) and
    # date and time(from <th>)
    for table_row in news_table.findAll('tr'):
        headlines = table_row.a.get_text()
        data_text = table_row.td.get_text().split()
        
        if len(data_text) == 1:
            time = data_text[0]
        else:
            date = data_text[0]
            time = data_text[1]
            
        company = file_name.split('_')[0]
        
        parsed_news.append([company, date, time, headlines])

In [15]:
parsed_news

[['fb',
  'Jan-04-19',
  '06:22PM',
  'Facebook, Apple, Netflix, Amazon and Alphabet Power Friday Gains'],
 ['fb',
  'Jan-04-19',
  '05:15PM',
  'Earnings Reports for the Week of Jan. 7-11 (BBBY, KBH)'],
 ['fb',
  'Jan-04-19',
  '04:55PM',
  'COLUMN-Commentary: 2019 will be the year of more state control'],
 ['fb', 'Jan-04-19', '04:53PM', '3 Tech Trends to Watch in 2019'],
 ['fb', 'Jan-04-19', '04:30PM', "Will Denver's tech growth continue in 2019?"],
 ['fb', 'Jan-04-19', '02:46PM', 'Facebook to Prioritize Video in 2019'],
 ['fb',
  'Jan-04-19',
  '01:52PM',
  'ScotteVest CEO: How a Facebook Passion Project Evolved into a New Business'],
 ['fb',
  'Jan-04-19',
  '01:47PM',
  'Millennials Load Up on Battered Apple, Facebook, GE'],
 ['fb',
  'Jan-04-19',
  '01:00PM',
  'FAANGs See a Weak Start to 2019: More Pain Ahead for ETFs?'],
 ['fb',
  'Jan-04-19',
  '12:18PM',
  'U[PDATE: Netflix shares soar 9% as Goldman adds stock to its Conviction List'],
 ['fb',
  'Jan-04-19',
  '12:11PM',
  'W

### Updating NLTK to identify financial sentiment

In [19]:
# New words and values for making lexicon read financial sentiment
new_words = {
    'crushes': 10,
    'beats': 5,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
}
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()
# Updating the vader lexicon
vader.lexicon.update(new_words)

## Reading the data into dataframe

In [21]:
columns = ['company', 'date', 'time', 'headline']

# Convert the list of lists into a DataFrame with above columns
scored_news = pd.DataFrame(parsed_news, columns=columns)

# Iterate through the headlines and get the polarity scores
scores = [vader.polarity_scores(headline) for headline in scored_news.headline.values]

# creating the polarity scores dataframe
scores_df = pd.DataFrame(scores)

# Join the DataFrames
scored_news = pd.concat([scored_news, scores_df], axis=1)

# Convert the date column from string to datetime
scored_news['date'] = pd.to_datetime(scored_news.date).dt.date